In [1]:
import sys
import numpy as np
sys.path.append("/Users/ruby/Astro/rlb_LWA")
import casa_utils
import pyuvdata

In [2]:
uvcal = casa_utils.cal_ms_to_uvcal(
    "/Users/ruby/Astro/LWA_data/LWA_data_20220307/20220307_175923_61MHz.bcal"
)

UVW orientation appears to be flipped, attempting to fix by changing conjugation of baselines.


Successful readonly open of default-locked table /Users/ruby/Astro/LWA_data/LWA_data_20220307/20220307_175923_61MHz.bcal: 13 columns, 352 rows
Successful readonly open of default-locked table /Users/ruby/Astro/LWA_data/LWA_data_20220307/20220307_175923_61MHz.ms/ANTENNA: 8 columns, 352 rows


telescope_location is not set. Using known values for OVRO-LWA.


In [3]:
uvd = pyuvdata.UVData()
uvd.read_uvfits("/Users/ruby/Astro/LWA_data/LWA_data_20220307/20220307_175923_61MHz_uncalib.uvfits")

In [4]:
# Initial flagging fraction
np.sum(uvd.flag_array)/np.size(uvd.flag_array)

0.5384045840844708

In [9]:
# Apply cal flags to uvd object:
for ant_ind in range(uvd.Nants_data):
    ant_name = uvd.antenna_names[ant_ind]
    cal_ant_names = np.array([uvcal.antenna_names[ind] for ind in uvcal.ant_array])
    uvcal_ant_ind = np.where(cal_ant_names == ant_name)[0][0]
    bl_inds_1 = np.where(uvd.ant_1_array == ant_ind)
    bl_inds_2 = np.where(uvd.ant_2_array == ant_ind)
    bl_inds = np.unique(np.concatenate((bl_inds_1, bl_inds_2), axis=None))
    for bl in bl_inds:
        uvd.flag_array[bl, 0, :, :] = uvd.flag_array[bl, 0, :, :] | np.repeat(
            np.max(uvcal.flag_array, axis=(1,3,4))[uvcal_ant_ind, :, np.newaxis],
            uvd.Npols,
            axis = 1
        )

In [6]:
# Final flagging fraction
np.sum(uvd.flag_array)/np.size(uvd.flag_array)

0.5726061518156065

In [7]:
uvd.write_uvfits("/Users/ruby/Astro/LWA_data/LWA_data_20220307/20220307_175923_61MHz_uncalib_transferred_flags.uvfits")

antnums_to_baseline: found antenna numbers > 256, using 2048 baseline indexing. Beware compatibility with CASA etc
Found antenna numbers > 256 in this data set. This is permitted by UVFITS standards, but may cause the `importuvfits` utility within CASA to crash. If attempting to use this data set in CASA, consider using the measurement set writer method (`write_ms`) instead.
